<a href="https://colab.research.google.com/github/vlopezma/SyS_25-1/blob/main/Taller_Laplace/Taller_Dashboard_Laplace_Valeria_Lopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#instalación de librerías
!pip install streamlit -q
!pip install --upgrade control -q
!pip install soundfile yt-dlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.3/578.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.8 MB/s eta 0:00:00


In [2]:
!apt install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!mkdir pages

In [5]:
%%writefile 📝_Taller_Laplace.py

import streamlit as st


st.set_page_config(
    page_title="Simulaciones - Transformada de Laplace - SyS",
    page_icon="📝",
)

# Título y descripción principal
st.title("📡 Taller 2 – Transformada de Laplace - Señales y Sistemas.")
st.markdown("##")
st.markdown("---")


st.markdown(r"""
Este simulador permite analizar el comportamiento dinámico de sistemas de segundo orden implementados como circuitos eléctricos **RLC** en configuración **serie o paralelo**, aplicando la **Transformada de Laplace**.

El objetivo es explorar las respuestas del sistema ante distintas condiciones de amortiguamiento:

- **Subamortiguado**: $0 < \zeta < 1$
- **Críticamente amortiguado**: $\zeta = 1$
- **Sobreamortiguado**: $\zeta > 1$
- **Inestable**: $\zeta < 0$

---

Para cada caso, el simulador mostrará:

- La **función de transferencia** del sistema:
  $$ H(s) = \frac{1}{LCs^2 + RCs + 1} $$
- Los **parámetros del sistema**:
  - Frecuencia natural no amortiguada: $\omega_n = \frac{1}{\sqrt{LC}}$
  - Factor de amortiguamiento: $\zeta = \frac{R}{2} \sqrt{\frac{C}{L}}$
  - Frecuencia natural amortiguada: $\omega_d = \omega_n \sqrt{1 - \zeta^2}$
- El **diagrama de polos y ceros**
- El **diagrama de Bode**
- La **respuesta en el tiempo** (a impulso, escalón, rampa)
- **Indicadores de desempeño**: tiempo de levantamiento ($t_r$), sobreimpulso máximo ($M_p$), tiempo de establecimiento ($t_s$), etc.

---

Este trabajo forma parte del **Taller 2** del curso de **Señales y Sistemas** (2025-1)
Valeria López Marín
1002653352
""")

Overwriting 📝_Taller_Laplace.py


In [31]:
%%writefile 📐_Simulaciones.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import TransferFunction, bode, step, impulse, lti

# Función de retroalimentación negativa
def feedback(sys1, sys2=1):
    num1, den1 = sys1.num, sys1.den
    if isinstance(sys2, (int, float)):
        num2, den2 = [sys2], [1]
    else:
        num2, den2 = sys2.num, sys2.den

    num_fb = np.polymul(num1, num2)
    den_fb = np.polymul(den1, den2)
    den_total = np.polyadd(den_fb, num_fb)
    return TransferFunction(num1, den_total)

st.set_page_config(
    page_title="🔁 Simulación Circuito RLC",
    page_icon="🔁",
    layout="wide"
)

st.title("Simulación Lazo Cerrado – Circuito RLC")

st.markdown("""
Selecciona el tipo de amortiguamiento y observa cómo responde el sistema eléctrico equivalente (RLC) en lazo cerrado. Se consideran condiciones iniciales cero.
""")

# Sidebar - Parámetros interactivos
st.sidebar.header("⚙️ Parámetros del Sistema")
tipo_respuesta = st.sidebar.selectbox(
    "Tipo de Respuesta",
    ["Subamortiguada", "Sobreamortiguada", "Amortiguamiento Crítico", "Inestable"]
)

wn = st.sidebar.slider("Frecuencia Natural ωₙ (rad/s)", 0.1, 20.0, 5.0)
if tipo_respuesta == "Subamortiguada":
    zeta = st.sidebar.slider("ζ", 0.0, 1.0, 0.5)
elif tipo_respuesta == "Sobreamortiguada":
    zeta = st.sidebar.slider("ζ", 1.0, 2.0, 1.2)
elif tipo_respuesta == "Amortiguamiento Crítico":
    zeta = 1.0
else:
    zeta = st.sidebar.slider("ζ", -1.0, 0.0, -0.1)

# Parámetros físicos
m = 1.0
k = wn**2
c = 2 * zeta * wn * m
L = m
R = c
C = 1 / k

num = [1]
den = [m, c, k]
sys = TransferFunction(num, den)
sys_fb = feedback(sys, 1)

# Mostrar parámetros
col1, col2, col3 = st.columns(3)
col1.metric("Masa (m)", f"{m:.2f} kg")
col2.metric("Amortiguador (c)", f"{c:.2f} Ns/m")
col3.metric("Resorte (k)", f"{k:.2f} N/m")

col1, col2, col3 = st.columns(3)
col1.metric("Inductor (L)", f"{L:.2f} H")
col2.metric("Resistor (R)", f"{R:.2f} Ω")
col3.metric("Capacitor (C)", f"{C:.4f} F")

# Respuesta al escalón
t = np.linspace(0, 10, 1000)
t_step, y_step = step(sys_fb, T=t)
st.markdown("### 📈 Respuesta al Escalón (Lazo Cerrado)")
fig1, ax1 = plt.subplots()
ax1.plot(t_step, y_step, label="Escalón")
ax1.set_xlabel("Tiempo (s)")
ax1.set_ylabel("Respuesta")
ax1.grid(True)
st.pyplot(fig1)

# Respuesta al impulso
t_impulse, y_impulse = impulse(sys_fb, T=t)
st.markdown("### 💥 Respuesta al Impulso")
fig2, ax2 = plt.subplots()
ax2.plot(t_impulse, y_impulse, label="Impulso", color='orange')
ax2.set_xlabel("Tiempo (s)")
ax2.set_ylabel("Respuesta")
ax2.grid(True)
st.pyplot(fig2)

# Respuesta a la rampa
ramp_input = t
_, y_ramp, _ = lti(num, den).output(ramp_input, t)
st.markdown("### 🔼 Respuesta a la Rampa")
fig3, ax3 = plt.subplots()
ax3.plot(t, y_ramp, label="Rampa", color='green')
ax3.set_xlabel("Tiempo (s)")
ax3.set_ylabel("Respuesta")
ax3.grid(True)
st.pyplot(fig3)

# Diagrama de polos y ceros
st.markdown("### ✳️ Mapa de Polos y Ceros")
poles = np.roots(den)
zeros = np.roots(num)
fig4, ax4 = plt.subplots()
ax4.scatter(np.real(poles), np.imag(poles), color='red', marker='x', label='Polos')
ax4.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', label='Ceros')
ax4.axhline(0, color='black', lw=1)
ax4.axvline(0, color='black', lw=1)
ax4.set_xlabel("Re")
ax4.set_ylabel("Im")
ax4.grid(True)
ax4.legend()
st.pyplot(fig4)

# Parámetros temporales si subamortiguado
if 0 < zeta < 1:
    wd = wn * np.sqrt(1 - zeta**2)
    tr = np.pi / wd
    tp = np.pi / wd
    Mp = np.exp(-zeta * np.pi / np.sqrt(1 - zeta**2))
    ts = 4 / (zeta * wn)

    st.markdown("### 📊 Parámetros Temporales")
    df = pd.DataFrame({
        "Parámetro": [
            "Tiempo de levantamiento (tr)",
            "Tiempo al pico (tp)",
            "Sobreimpulso máximo (Mp)",
            "Tiempo de establecimiento (ts)"
        ],
        "Valor": [
            f"{tr:.3f} s",
            f"{tp:.3f} s",
            f"{Mp*100:.2f} %",
            f"{ts:.3f} s"
        ]
    })
    st.table(df)

# Diagrama de Bode
st.markdown("### 📉 Diagrama de Bode")
w, mag, phase = bode(sys_fb)
fig5, (ax5, ax6) = plt.subplots(2, 1, figsize=(8, 6))
ax5.semilogx(w, mag)
ax5.set_ylabel("Magnitud (dB)")
ax5.grid(True, which="both")

ax6.semilogx(w, phase)
ax6.set_xlabel("Frecuencia (rad/s)")
ax6.set_ylabel("Fase (°)")
ax6.grid(True, which="both")

st.pyplot(fig5)


Writing 📐_Simulaciones.py


In [32]:
!mv 📐_Simulaciones.py pages/

In [33]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 📝_Taller_Laplace.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunne                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  l
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-07-14 23:20:18--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64 [following]
--2025-07-14 23:20:18--  https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/37d2bad8-a2ed-4b93-8139-cbb15162d81d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-15T00%3A12%3A52Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-14T2

In [34]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")

Digite (1) para finalizar la ejecución del Dashboard: 1
El proceso de Streamlit ha sido finalizado.
